In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from  xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier,CatBoostRegressor
from xgboost import XGBClassifier,XGBRFClassifier

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.over_sampling import SMOTE

In [3]:
CATEGORIES = {
    'Men Tshirts': 5,
    'Sarees': 10,
    'Kurtis': 9,
    'Women Tshirts': 8,
    'Women Tops & Tunics': 10
}
cat_map=dict(zip(CATEGORIES.keys(),range(5)))
cat_inv_map=dict(zip(range(5),CATEGORIES.keys()))

In [4]:
clip_train=pd.read_csv("/kaggle/input/vit-embeddings/train_clip.csv")
clip_test=pd.read_csv("/kaggle/input/vit-embeddings/test_clip.csv")
clip_train.columns = [f"clip_{i}" for i in range(clip_train.shape[1])]
clip_test.columns = [f"clip_{i}" for i in range(clip_test.shape[1])]


In [5]:
clip_train_ext=pd.read_csv("/kaggle/input/clip-embs/train_clip_extreme.csv")
clip_test_ext=pd.read_csv("/kaggle/input/clip-embs/test_clip_extreme.csv")
clip_train_ext.columns = [f"clip_ext_{i}" for i in range(clip_train_ext.shape[1])]
clip_test_ext.columns = [f"clip_ext_{i}" for i in range(clip_test_ext.shape[1])]


In [6]:
clip_train_neg=pd.read_csv("/kaggle/input/clip-embs/train_clip_neg.csv")
clip_test_neg=pd.read_csv("/kaggle/input/clip-embs/test_clip_neg.csv")
clip_train_neg.columns = [f"clip_neg_{i}" for i in range(clip_train_neg.shape[1])]
clip_test_neg.columns = [f"clip_neg_{i}" for i in range(clip_test_neg.shape[1])]


In [7]:
train_df=pd.read_csv("/kaggle/input/meesho-train-pre-pro/pre_processed_train.csv")
test_df=pd.read_csv("/kaggle/input/visual-taxonomy/test.csv")
train_df=train_df.drop(["Unnamed: 0","image_path","ocr_text"],axis=1)
vit_train=np.load("/kaggle/input/meesho-augmentation/orignal_train_features.npy")
vit_test=np.load("/kaggle/input/meesho-augmentation/orignal_test_features.npy")

vit_train=pd.DataFrame(vit_train,columns=[f"vit_emb_{i}" for i in range(vit_train.shape[1])])
vit_test=pd.DataFrame(vit_test,columns=[f"vit_emb_{i}" for i in range(vit_test.shape[1])])

In [8]:
vit_black_train=np.load("/kaggle/input/meesho-vit-black-and-white/train_features.npy")
vit_black_test=np.load("/kaggle/input/meesho-vit-black-and-white/test_features.npy")
vit_black_train=pd.DataFrame(vit_black_train,columns=[f"vit_black_emb_{i}" for i in range(vit_black_train.shape[1])])
vit_black_test=pd.DataFrame(vit_black_test,columns=[f"vit_black_emb_{i}" for i in range(vit_black_test.shape[1])])

In [9]:
train_df=pd.concat([train_df,vit_train,clip_train,vit_black_train,clip_train_ext,clip_train_neg],axis=1)
test_df=pd.concat([test_df,vit_test,clip_test,vit_black_test,clip_test_ext,clip_test_neg],axis=1)

In [10]:
labels=['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6',
       'attr_7', 'attr_8', 'attr_9', 'attr_10']

In [11]:
final_predictions=test_df[["id","Category"]].copy()
final_predictions["len"]=test_df.Category.map(CATEGORIES)
start = 0
end = 10
for i in range(start,end):
    final_predictions[f"attr_{i+1}"]=0

In [12]:
attr_vals=[]
attr_dicts=[]
for i in range(1,11):
    vals=(train_df[f"attr_{i}"].dropna().unique())
#     vals=(train[col].unique())
    attr_vals.append(vals)
    attr_dicts.append(dict(zip(vals,range(0,len(vals)))))
attr_dicts
for i in range(10):
    train_df[f"attr_{i+1}"]=train_df[f"attr_{i+1}"].map(attr_dicts[i])
    

train_df["Category"]=train_df["Category"].map(cat_map)
test_df["Category"]=test_df["Category"].map(cat_map)

In [13]:
from sklearn.model_selection import StratifiedKFold
from statistics import mode
cat_cv=0

for cat in range(5):
    print("Catergory: ", cat_inv_map[cat])
    attr_cv=0
    for attr_idx, attr in enumerate(labels[start:end]):
        
        print(f"Attribute: {attr}")
        train_df1 = train_df[train_df.Category==cat].dropna(subset=[attr])
        if (len(train_df1)==0):
            continue
            
        X = train_df1.drop(labels, axis=1)
        X = X.drop(["len", "id"], axis=1)
        y = train_df1[attr]

        # Do feature selection once
        initial_model = LGBMClassifier(
            class_weight="balanced",
            n_jobs=4,
            verbose=-1,
            n_estimators=70,
            extra_trees=True,

        )
        initial_model.fit(X, y)
        feature_importance = pd.Series(initial_model.feature_importances_, index=X.columns)
        n_features_to_keep = X.shape[1] - 2100  
        top_features = feature_importance.nlargest(n_features_to_keep).index
        X = X[top_features]        
        # Initialize StratifiedKFold
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4)
        
        # For storing all fold predictions
        test_predictions = np.zeros((len(test_df[test_df.Category == cat]), 5))
        fold_scores_macro = []
        fold_scores_micro = []

        # Run cross-validation
        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            print(f"Fold {fold+1}",end=" ")
            
            # Split data
            x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            # Apply SMOTE
#             smote = SMOTE(random_state=42)
#             x_train, y_train = smote.fit_resample(x_train, y_train)
            
            # Train model
            model = LGBMClassifier(
                class_weight="balanced",
                n_jobs=4,
                early_stopping_rounds=20,
                extra_trees=True,
                verbose=-1,
            )
            model.fit(x_train, y_train, eval_set=[(x_val, y_val)])
            
            # Evaluate on validation set
            val_pred = model.predict(x_val)
            fold_scores_macro.append(f1_score(y_val, val_pred, average='macro'))
            fold_scores_micro.append(f1_score(y_val, val_pred, average='micro'))
            
            # Predict on test set
            test_data = test_df[test_df.Category == cat].drop("id", axis=1)
            test_data = test_data[top_features]
            test_predictions[:, fold] = model.predict(test_data)    

        # Print scores for all folds
        
        macro=(np.mean(fold_scores_macro))
        micro=(np.mean(fold_scores_micro))
        print()
        print(f"Average F1 macro: ", macro)
        print(f"Average F1 micro: ", micro)
        hm=((2*macro*micro)/(micro+macro))
        print("harmonic mean",hm)
        attr_cv+=hm
        # Get mode of predictions for each test sample
        final_pred = [mode(preds) for preds in test_predictions]
        final_predictions.loc[test_df.Category == cat, attr] = final_pred
    print("cat_cv: " ,attr_cv/CATEGORIES[cat_inv_map[cat]])
    cat_cv+=attr_cv/CATEGORIES[cat_inv_map[cat]]
print("final_cv: ", cat_cv/5)

Catergory:  Men Tshirts
Attribute: attr_1
Fold 1 Fold 2 Fold 3 Fold 4 Fold 5 
Average F1 macro:  0.7720685752809608
Average F1 micro:  0.7675540765391015
harmonic mean 0.7698047071782054
Attribute: attr_2
Fold 1 Fold 2 Fold 3 Fold 4 Fold 5 
Average F1 macro:  0.9939782621110347
Average F1 micro:  0.9939783145111489
harmonic mean 0.9939782883110911
Attribute: attr_3
Fold 1 Fold 2 Fold 3 Fold 4 Fold 5 
Average F1 macro:  0.9711031980554132
Average F1 micro:  0.9713342006166356
harmonic mean 0.9712186856001421
Attribute: attr_4
Fold 1 Fold 2 Fold 3 Fold 4 Fold 5 
Average F1 macro:  0.8599369426219579
Average F1 micro:  0.904186131980126
harmonic mean 0.8815065899770588
Attribute: attr_5
Fold 1 Fold 2 Fold 3 Fold 4 Fold 5 
Average F1 macro:  0.975085573269143
Average F1 micro:  0.9948140944011419
harmonic mean 0.9848510433859526
Attribute: attr_6
Attribute: attr_7
Attribute: attr_8
Attribute: attr_9
Attribute: attr_10
cat_cv:  0.92027186289049
Catergory:  Sarees
Attribute: attr_1
Fold 1 Fo

In [14]:
for i in range(start,end):
    final_predictions[f"attr_{i+1}"]=final_predictions[f"attr_{i+1}"].map(dict(zip(range(len(attr_vals[i])),attr_vals[i])))

In [15]:
final_predictions.to_csv(f"sub_{start}_{end}.csv")